In [2]:
# ======================
# 1. 라이브러리 및 패키지 임포트
# ======================

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [3]:
# ======================
# 2. 각 데이터 불러오기 (gold_data 폴더에서)
# ======================

print("gold_data 폴더에서 데이터 로딩 시작...")

# 데이터 경로
data_path = "./gold_data/"

# 각 파일별로 데이터 로딩
data_files = {}

# 01번부터 13번까지 파일 로딩
for i in range(1, 14):
    file_num = f"{i:02d}"  # 01, 02, 03, ... 13
    try:
        file_path = f"{data_path}{file_num}*.csv"
        import glob
        matching_files = glob.glob(file_path)
        
        if matching_files:
            # 첫 번째 매칭 파일 사용
            df = pd.read_csv(matching_files[0], index_col=0, parse_dates=True)
            data_files[f"data_{file_num}"] = df
            print(f"{file_num}번 파일 로딩 완료: {df.shape}")
        else:
            print(f"{file_num}번 파일을 찾을 수 없습니다.")
            
    except Exception as e:
        print(f"{file_num}번 파일 로딩 실패: {e}")

print(f"총 {len(data_files)}개 파일 로딩 완료")

# 로딩된 데이터 확인
for key, df in data_files.items():
    print(f"{key}: {df.shape}, 컬럼: {list(df.columns)}")

gold_data 폴더에서 데이터 로딩 시작...
01번 파일 로딩 완료: (186, 5)
02번 파일 로딩 완료: (186, 1)
03번 파일 로딩 완료: (186, 1)
04번 파일 로딩 완료: (186, 2)
05번 파일 로딩 완료: (186, 10)
06번 파일 로딩 완료: (186, 2)
07번 파일을 찾을 수 없습니다.
08번 파일 로딩 완료: (186, 5)
09번 파일 로딩 완료: (186, 6)
10번 파일 로딩 완료: (186, 5)
11번 파일 로딩 완료: (186, 5)
12번 파일 로딩 완료: (186, 5)
13번 파일 로딩 완료: (186, 1)
총 12개 파일 로딩 완료
data_01: (186, 5), 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
data_02: (186, 1), 컬럼: ['M2_Money_Supply_Billions_USD']
data_03: (186, 1), 컬럼: ['CPIAUCNS']
data_04: (186, 2), 컬럼: ['Close', 'Volume']
data_05: (186, 10), 컬럼: ['Gold_Reserves_United_States', 'Gold_Reserves_Germany', 'Gold_Reserves_Italy', 'Gold_Reserves_France', 'Gold_Reserves_Russia', 'Gold_Reserves_China', 'Gold_Reserves_Switzerland', 'Gold_Reserves_Japan', 'Gold_Reserves_India', 'Gold_Reserves_Netherlands']
data_06: (186, 2), 컬럼: ['USD_CNY_Rate', 'USD_EUR_Rate']
data_08: (186, 5), 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
data_09: (186, 6), 컬럼: ['Price', 'Close', 'High', 'Low', 'O

In [4]:
# ======================
# 2-1. 필요한 컬럼만 추출 (CLOSE, Volume)
# ======================

print("필요한 컬럼 추출 중...")

extracted_data = {}

# 컬럼 찾기 함수
def find_column(df, target_names):
    """대소문자 구분 없이 컬럼 찾기"""
    for target in target_names:
        for col in df.columns:
            if col.upper() == target.upper():
                return col
    return None

# 각 파일별로 필요한 컬럼 추출
for i in range(1, 14):
    file_num = f"{i:02d}"
    data_key = f"data_{file_num}"
    
    if data_key in data_files:
        df = data_files[data_key]
        print(f"\n{file_num}번 파일 컬럼: {list(df.columns)}")
        
        # 파일별 컬럼 추출 규칙
        if file_num == "01":  # 종속변수 - CLOSE만
            close_col = find_column(df, ['CLOSE', 'Close', 'close'])
            if close_col:
                extracted_data[f"feature_{file_num}"] = df[close_col]
                print(f"{file_num}번: {close_col} 추출 (종속변수)")
            else:
                # CLOSE가 없으면 첫 번째 컬럼 사용
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}번: {first_col} 추출 (CLOSE 없어서 첫번째 컬럼 사용)")
                
        elif file_num == "04":  # Volume 컬럼에서 'Volume'
            volume_col = find_column(df, ['Volume', 'VOLUME', 'volume'])
            if volume_col:
                extracted_data[f"feature_{file_num}"] = df[volume_col]
                print(f"{file_num}번: {volume_col} 추출")
            else:
                # Volume이 없으면 첫 번째 컬럼 사용
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}번: {first_col} 추출 (Volume 없어서 첫번째 컬럼 사용)")
                
        elif file_num in ["05", "06"]:  # 05, 06번은 모든 컬럼 포함
            for col in df.columns:
                feature_name = f"feature_{file_num}_{col}"
                extracted_data[feature_name] = df[col]
            print(f"{file_num}번: 모든 컬럼 추출 ({len(df.columns)}개)")
                
        elif file_num in ["08", "09", "10", "11"]:  # CLOSE만
            close_col = find_column(df, ['CLOSE', 'Close', 'close'])
            if close_col:
                extracted_data[f"feature_{file_num}"] = df[close_col]
                print(f"{file_num}번: {close_col} 추출")
            else:
                # CLOSE가 없으면 첫 번째 컬럼 사용
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}번: {first_col} 추출 (CLOSE 없어서 첫번째 컬럼 사용)")
                
        else:  # 나머지는 CLOSE 우선, 없으면 첫 번째 컬럼
            close_col = find_column(df, ['CLOSE', 'Close', 'close'])
            if close_col:
                extracted_data[f"feature_{file_num}"] = df[close_col]
                print(f"{file_num}번: {close_col} 추출")
            else:
                # CLOSE가 없으면 첫 번째 컬럼 사용
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}번: {first_col} 추출 (CLOSE 없어서 첫번째 컬럼 사용)")

print(f"\n추출된 피쳐 수: {len(extracted_data)}개")

# 추출된 데이터 미리보기
for key, series in extracted_data.items():
    print(f"{key}: {len(series)}개 데이터, 기간: {series.index[0]} ~ {series.index[-1]}")

필요한 컬럼 추출 중...

01번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
01번: Close 추출 (종속변수)

02번 파일 컬럼: ['M2_Money_Supply_Billions_USD']
02번: M2_Money_Supply_Billions_USD 추출 (CLOSE 없어서 첫번째 컬럼 사용)

03번 파일 컬럼: ['CPIAUCNS']
03번: CPIAUCNS 추출 (CLOSE 없어서 첫번째 컬럼 사용)

04번 파일 컬럼: ['Close', 'Volume']
04번: Volume 추출

05번 파일 컬럼: ['Gold_Reserves_United_States', 'Gold_Reserves_Germany', 'Gold_Reserves_Italy', 'Gold_Reserves_France', 'Gold_Reserves_Russia', 'Gold_Reserves_China', 'Gold_Reserves_Switzerland', 'Gold_Reserves_Japan', 'Gold_Reserves_India', 'Gold_Reserves_Netherlands']
05번: 모든 컬럼 추출 (10개)

06번 파일 컬럼: ['USD_CNY_Rate', 'USD_EUR_Rate']
06번: 모든 컬럼 추출 (2개)

08번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
08번: Close 추출

09번 파일 컬럼: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']
09번: Close 추출

10번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
10번: Close 추출

11번 파일 컬럼: ['Open', 'High', 'Low', 'Close', 'Volume']
11번: Close 추출

12번 파일 컬럼: ['close', 'open', 'high', 'low', 'change_pct']

In [6]:
# ======================
# 2-2. 모든 데이터 날짜 형식 통일 (YYYY-MM)
# ======================

print("모든 데이터 날짜 형식 통일 중... (YYYY-MM 문자열 형태)")

# 날짜 인덱스를 YYYY-MM 문자열로 변환하는 함수
def convert_to_yyyymm_string(series, series_name):
    """모든 형태의 날짜 인덱스를 YYYY-MM 문자열로 변환"""
    try:
        print(f"  {series_name} 변환 시작...")
        print(f"    원본 인덱스 타입: {type(series.index)}")
        
        # 이미 문자열이고 YYYY-MM 형식인지 확인
        if isinstance(series.index[0], str) and len(series.index[0]) == 7 and '-' in series.index[0]:
            print(f"    이미 YYYY-MM 문자열 형식입니다.")
            return series
        
        # DatetimeIndex가 아닌 경우 변환
        if not isinstance(series.index, pd.DatetimeIndex):
            print(f"    DatetimeIndex로 변환 중...")
            series.index = pd.to_datetime(series.index, utc=True)
        
        # timezone 제거
        if series.index.tz is not None:
            print(f"    timezone 제거 중...")
            series.index = series.index.tz_convert(None)
        
        # Period로 변환하여 월 단위로 집계
        print(f"    월별 집계 중...")
        period_index = series.index.to_period('M')
        
        # 같은 월에 여러 데이터가 있으면 평균값 사용
        if len(period_index.unique()) < len(period_index):
            print(f"    같은 월에 중복 데이터 있음. 평균값으로 집계...")
            temp_df = pd.DataFrame({'value': series.values, 'period': period_index})
            monthly_data = temp_df.groupby('period')['value'].mean()
        else:
            monthly_data = pd.Series(series.values, index=period_index)
        
        # Period를 YYYY-MM 문자열로 변환
        monthly_data.index = monthly_data.index.astype(str)
        monthly_data.name = series_name
        
        print(f"    변환 완료: {len(monthly_data)}개, {monthly_data.index[0]} ~ {monthly_data.index[-1]}")
        return monthly_data
        
    except Exception as e:
        print(f"    {series_name} 변환 실패: {e}")
        return series

# 모든 extracted_data를 YYYY-MM 문자열 형식으로 변환
print(f"\n총 {len(extracted_data)}개 데이터 처리 시작:")

standardized_data = {}

for key, series in extracted_data.items():
    print(f"\n{key} 처리 중...")
    standardized_series = convert_to_yyyymm_string(series.copy(), key)
    standardized_data[key] = standardized_series

# extracted_data 업데이트
extracted_data = standardized_data

print(f"\n 모든 데이터 변환 완료!")

# 변환된 데이터 정보 확인
print(f"\n변환된 데이터 정보:")
for key, series in extracted_data.items():
    print(f"  {key}: {len(series)}개, {series.index[0]} ~ {series.index[-1]} (YYYY-MM)")

# 공통 날짜 범위 계산
try:
    valid_series = []
    for series in extracted_data.values():
        if len(series) > 0:
            valid_series.append(series)
    
    if valid_series:
        # 모든 시작/끝 날짜 수집 (이제 모두 문자열)
        all_start_dates = [series.index[0] for series in valid_series]
        all_end_dates = [series.index[-1] for series in valid_series]
        
        # 문자열 정렬로 공통 범위 찾기
        common_start = max(all_start_dates)  # 가장 늦은 시작
        common_end = min(all_end_dates)      # 가장 이른 종료
        
        print(f"\n 모든 데이터의 공통 기간: {common_start} ~ {common_end}")
        
        # 공통 기간 내 월 수 계산
        start_year, start_month = map(int, common_start.split('-'))
        end_year, end_month = map(int, common_end.split('-'))
        months_count = (end_year - start_year) * 12 + (end_month - start_month) + 1
        print(f" 공통 기간 내 월 수: {months_count}개월")
        
        # 각 데이터가 공통 기간에서 몇 개의 데이터를 가지는지 확인
        print(f"\n각 데이터의 공통 기간 내 데이터 개수:")
        for key, series in extracted_data.items():
            mask = (series.index >= common_start) & (series.index <= common_end)
            common_data_count = mask.sum()
            print(f"  {key}: {common_data_count}개")
            
    else:
        print(f"\n 유효한 날짜 데이터가 없습니다.")
        
except Exception as e:
    print(f"\n공통 날짜 범위 계산 실패: {e}")

print(f"\nextracted_data 업데이트 완료")

모든 데이터 날짜 형식 통일 중... (YYYY-MM 문자열 형태)

총 22개 데이터 처리 시작:

feature_01 처리 중...
  feature_01 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_02 처리 중...
  feature_02 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_03 처리 중...
  feature_03 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_04 처리 중...
  feature_04 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_05_Gold_Reserves_United_States 처리 중...
  feature_05_Gold_Reserves_United_States 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_05_Gold_Reserves_Germany 처리 중...
  feature_05_Gold_Reserves_Germany 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    이미 YYYY-MM 문자열 형식입니다.

feature_05_Gold_Reserves_Italy 처리 중...
  feature_05_Gold_Reserves_Italy 변환 시작...
    원본 인덱스 타입: <cl

In [7]:
# feature_04와 feature_11 날짜 변환 문제 분석

print("=== 날짜 변환 문제 분석 ===")

# 원본 data_04 상세 확인
if 'data_04' in data_files:
    df_04 = data_files['data_04']
    print(f"\ndata_04 원본 데이터 분석:")
    print(f"전체 크기: {df_04.shape}")
    print(f"인덱스 타입: {type(df_04.index)}")
    
    # Volume 컬럼 확인
    volume_col = None
    for col in df_04.columns:
        if col.upper() == 'VOLUME':
            volume_col = col
            break
    
    if volume_col:
        print(f"Volume 컬럼 발견: {volume_col}")
        volume_series = df_04[volume_col]
        print(f"Volume 데이터 크기: {len(volume_series)}")
        print(f"결측치 개수: {volume_series.isnull().sum()}")
        
        # 마지막 10개 데이터 확인
        print(f"\n마지막 10개 데이터:")
        print(volume_series.tail(10))
        
        # 날짜 변환 테스트
        print(f"\n날짜 변환 테스트:")
        try:
            # datetime 변환
            datetime_index = pd.to_datetime(volume_series.index, errors='coerce')
            print(f"datetime 변환 성공: {len(datetime_index)}개")
            print(f"NaT 개수: {datetime_index.isna().sum()}")
            
            # 유효한 데이터만 선택
            valid_mask = ~datetime_index.isna()
            valid_series = volume_series[valid_mask]
            valid_datetime = datetime_index[valid_mask]
            
            print(f"유효한 데이터: {len(valid_series)}개")
            print(f"유효한 날짜 범위: {valid_datetime[0]} ~ {valid_datetime[-1]}")
            
            # Period 변환 테스트
            period_index = valid_datetime.to_period('M')
            print(f"Period 변환 후: {len(period_index)}개")
            print(f"고유한 Period: {len(period_index.unique())}개")
            
            # 월별 그룹화 테스트
            temp_df = pd.DataFrame({'value': valid_series.values, 'period': period_index})
            monthly_avg = temp_df.groupby('period')['value'].mean()
            print(f"월별 평균 계산 후: {len(monthly_avg)}개")
            print(f"마지막 5개 월:")
            print(monthly_avg.tail())
            
        except Exception as e:
            print(f"날짜 변환 실패: {e}")
    else:
        print("Volume 컬럼을 찾을 수 없습니다.")

# 원본 data_11도 확인
if 'data_11' in data_files:
    df_11 = data_files['data_11']
    print(f"\n\ndata_11 원본 데이터 분석:")
    print(f"전체 크기: {df_11.shape}")
    
    # Close 컬럼 확인
    close_col = None
    for col in df_11.columns:
        if col.upper() == 'CLOSE':
            close_col = col
            break
    
    if close_col:
        print(f"Close 컬럼 발견: {close_col}")
        close_series = df_11[close_col]
        print(f"Close 데이터 크기: {len(close_series)}")
        print(f"결측치 개수: {close_series.isnull().sum()}")
        print(f"마지막 10개 데이터:")
        print(close_series.tail(10))

print(f"\n=== 결론 ===")
print("원본 데이터는 2025-06까지 있지만, 날짜 변환 과정에서 일부 데이터가 손실되고 있습니다.")
print("standardize_date_index 함수를 다시 확인해야 합니다.")

=== 날짜 변환 문제 분석 ===

data_04 원본 데이터 분석:
전체 크기: (186, 2)
인덱스 타입: <class 'pandas.core.indexes.base.Index'>
Volume 컬럼 발견: Volume
Volume 데이터 크기: 186
결측치 개수: 0

마지막 10개 데이터:
Date
2024-09-30 00:00:00-04:00    130608100
2024-10-31 00:00:00-04:00    145957700
2024-11-30 00:00:00-05:00    152567400
2024-12-31 00:00:00-05:00    115478400
2025-01-31 00:00:00-05:00    141384500
2025-02-28 00:00:00-05:00    178992600
2025-03-31 00:00:00-04:00    181005700
2025-04-30 00:00:00-04:00    354296400
2025-05-31 00:00:00-04:00    235215100
2025-06-30 00:00:00-04:00    191373000
Name: Volume, dtype: int64

날짜 변환 테스트:
datetime 변환 성공: 186개
NaT 개수: 124
유효한 데이터: 62개
유효한 날짜 범위: 2010-01-31 00:00:00-05:00 ~ 2025-02-28 00:00:00-05:00
Period 변환 후: 62개
고유한 Period: 62개
월별 평균 계산 후: 62개
마지막 5개 월:
period
2024-02    118520300.0
2024-11    152567400.0
2024-12    115478400.0
2025-01    141384500.0
2025-02    178992600.0
Freq: M, Name: value, dtype: float64


data_11 원본 데이터 분석:
전체 크기: (186, 5)
Close 컬럼 발견: Close
Close 데이터 크기

In [9]:
# ======================
# 3. 데이터 통합 및 병합
# ======================

print("데이터 통합 중... (YYYY-MM 문자열 인덱스)")

# 추출된 데이터들을 DataFrame으로 병합
all_features = []
feature_names = []

# 각 데이터의 정보 확인
print("\n추출된 데이터 정보:")
for key, series in extracted_data.items():
    print(f"{key}: {len(series)}개, {series.index[0]} ~ {series.index[-1]}")

# 피쳐 이름 매핑 (각 파일에 맞는 의미있는 이름)
feature_name_mapping = {
    'feature_01': 'Gold_Price',           # 금시세 (종속변수)
    'feature_02': 'US_M2_Money_Supply',   # 미국 M2통화량
    'feature_03': 'US_CPI',               # 미국 소비자물가지수
    'feature_04': 'Gold_Volume',          # 금 거래량
    'feature_05': 'Gold_Reserves',        # 금 보유량 (05번 파일의 모든 컬럼들)
    'feature_06': 'Exchange_Rates',       # 환율 관련 (06번 파일의 모든 컬럼들)
    'feature_07': 'Silver_Price',         # 은 시세
    'feature_08': 'Dollar_Index',         # 달러지수
    'feature_09': 'SP500',                # S&P 500
    'feature_10': 'NASDAQ',               # 나스닥
    'feature_11': 'Bond_Rate',            # 채권금리
    'feature_12': 'VIX',                  # VIX 공포지수
    'feature_13': 'Additional_Economic'   # 추가 경제지표
}

# 모든 추출된 피쳐를 정렬하여 추가
# 01번부터 13번까지 순서 유지
sorted_keys = sorted(extracted_data.keys(), key=lambda x: (
    int(x.split('_')[1]), x.split('_')[2] if len(x.split('_')) > 2 else ''
))

print(f"\n정렬된 키 순서: {sorted_keys}")

for feature_key in sorted_keys:
    series = extracted_data[feature_key]
    
    # 의미있는 피쳐 이름 생성
    if feature_key.startswith('feature_05_'):
        # 05번 파일의 각 컬럼
        suffix = feature_key.replace('feature_05_', '')
        meaningful_name = f'Gold_Reserves_{suffix}'
    elif feature_key.startswith('feature_06_'):
        # 06번 파일의 각 컬럼
        suffix = feature_key.replace('feature_06_', '')
        meaningful_name = f'Exchange_Rate_{suffix}'
    else:
        # 단일 컬럼 파일들
        base_key = '_'.join(feature_key.split('_')[:2])  # feature_01, feature_02 등
        meaningful_name = feature_name_mapping.get(base_key, feature_key)
    
    series.name = meaningful_name
    all_features.append(series)
    feature_names.append(meaningful_name)
    print(f"{feature_key} -> {meaningful_name} 추가됨 (크기: {len(series)})")

# 모든 피쳐를 하나의 DataFrame으로 병합
if all_features:
    print(f"\n{len(all_features)}개 시리즈 병합 시작...")
    
    # 각 시리즈의 날짜 범위 확인
    print("\n각 피쳐의 날짜 범위:")
    for feature in all_features:
        print(f"  {feature.name}: {feature.index[0]} ~ {feature.index[-1]} ({len(feature)}개)")
    
    # 공통 날짜 범위 찾기 (YYYY-MM 문자열 기준)
    print("\n공통 날짜 범위 계산 중...")
    
    # 모든 피쳐의 시작/끝 날짜 수집 (문자열)
    start_dates = [feature.index[0] for feature in all_features if len(feature) > 0]
    end_dates = [feature.index[-1] for feature in all_features if len(feature) > 0]
    
    if start_dates and end_dates:
        common_start = max(start_dates)  # 가장 늦은 시작 날짜 (문자열 정렬)
        common_end = min(end_dates)      # 가장 이른 종료 날짜 (문자열 정렬)
        
        print(f"공통 기간: {common_start} ~ {common_end}")
        
        # 공통 기간으로 모든 시리즈 필터링
        filtered_features = []
        for feature in all_features:
            # 공통 기간 내의 데이터만 선택 (문자열 비교)
            mask = (feature.index >= common_start) & (feature.index <= common_end)
            filtered_feature = feature[mask]
            
            if len(filtered_feature) > 0:
                filtered_features.append(filtered_feature)
                print(f"  {feature.name}: {len(filtered_feature)}개 데이터 (필터링 후)")
            else:
                print(f"  {feature.name}: 공통 기간에 데이터 없음 - 제외")
        
        # 필터링된 피쳐들로 병합 (outer join)
        if filtered_features:
            combined_data = pd.concat(filtered_features, axis=1, join='outer')
            
            print(f"\n병합 후 크기: {combined_data.shape}")
            print(f"병합 후 기간: {combined_data.index[0]} ~ {combined_data.index[-1]}")
            
            # 결측치 확인
            print(f"\n병합 후 결측치 현황:")
            missing_counts = combined_data.isnull().sum()
            total_missing = missing_counts.sum()
            
            if total_missing > 0:
                print(f"총 결측치: {total_missing}개")
                for col, missing in missing_counts.items():
                    if missing > 0:
                        missing_pct = (missing / len(combined_data)) * 100
                        print(f"  {col}: {missing}개 ({missing_pct:.1f}%)")
            else:
                print("✅ 결측치가 없습니다!")
            
            print("데이터 통합 완료")
            print(f"최종 통합 데이터 크기: {combined_data.shape}")
            print(f"최종 기간: {combined_data.index[0]} ~ {combined_data.index[-1]}")
            
            # 인덱스를 YYYY-MM 형식으로 설정
            combined_data.index.name = 'Date_YYYY_MM'
            
            # 원본 통합 데이터 저장 (스케일링 전)
            raw_filename = "gold_data_raw_combined.csv"
            combined_data.to_csv(raw_filename, encoding='utf-8-sig')
            print(f"원본 통합 데이터 저장: {raw_filename}")
            
            # 컬럼 정보 출력
            print("\n최종 피쳐 목록:")
            for i, col in enumerate(combined_data.columns, 1):
                print(f"  {i:2d}. {col}")
            
            # 각 컬럼의 유효 데이터 개수 확인
            print(f"\n각 컬럼별 유효 데이터 개수:")
            for col in combined_data.columns:
                valid_count = combined_data[col].count()
                total_count = len(combined_data)
                coverage = valid_count/total_count*100
                print(f"  {col}: {valid_count}/{total_count} ({coverage:.1f}%)")
            
            print(f"\n데이터 샘플 (처음 5행):")
            print(combined_data.head())
            
            print(f"\n데이터 샘플 (마지막 5행):")
            print(combined_data.tail())
            
        else:
            print("공통 기간에 유효한 데이터가 없습니다.")
            combined_data = pd.DataFrame()
    else:
        print("유효한 날짜 범위를 찾을 수 없습니다.")
        combined_data = pd.DataFrame()
    
else:
    print("병합할 데이터가 없습니다.")
    combined_data = pd.DataFrame()

데이터 통합 중... (YYYY-MM 문자열 인덱스)

추출된 데이터 정보:
feature_01: 186개, 2010-01 ~ 2025-06
feature_02: 186개, 2010-01 ~ 2025-06
feature_03: 186개, 2010-01 ~ 2025-06
feature_04: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_United_States: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Germany: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Italy: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_France: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Russia: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_China: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Switzerland: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Japan: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_India: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Netherlands: 186개, 2010-01 ~ 2025-06
feature_06_USD_CNY_Rate: 186개, 2010-01 ~ 2025-06
feature_06_USD_EUR_Rate: 186개, 2010-01 ~ 2025-06
feature_08: 186개, 2010-01 ~ 2025-06
feature_09: 186개, 2010-01 ~ 2025-06
feature_10: 186개, 2010-01 ~ 2025-06
feature_11: 186개, 2010-

In [10]:
# ======================
# 4. 결측치 처리 및 보간
# ======================

print("🔄 결측치 처리 중...")

if not combined_data.empty:
    # 결측치 확인
    print(" 결측치 현황:")
    missing_info = combined_data.isnull().sum()
    total_missing = missing_info.sum()
    
    print(f"전체 데이터 크기: {combined_data.shape}")
    print(f"총 결측치: {total_missing}개")
    
    if total_missing > 0:
        print("\n각 컬럼별 결측치:")
        for col, missing_count in missing_info.items():
            if missing_count > 0:
                missing_pct = (missing_count / len(combined_data)) * 100
                print(f"  {col}: {missing_count}개 ({missing_pct:.1f}%)")
        
        # 결측치가 너무 많은 컬럼은 제거 (50% 이상 결측치)
        high_missing_cols = []
        for col, missing_count in missing_info.items():
            missing_pct = (missing_count / len(combined_data)) * 100
            if missing_pct > 50:
                high_missing_cols.append(col)
        
        if high_missing_cols:
            print(f"\n 결측치가 50% 이상인 컬럼 제거: {high_missing_cols}")
            combined_data = combined_data.drop(columns=high_missing_cols)
            print(f"제거 후 데이터 크기: {combined_data.shape}")
        
        # 선형 보간법으로 결측치 처리
        print("\n 선형 보간법 적용 중...")
        data_interpolated = combined_data.interpolate(method='linear', limit_direction='both')
        
        # 여전히 남은 결측치는 앞뒤 값으로 채우기
        print(" 앞뒤 값으로 채우기...")
        data_filled = data_interpolated.fillna(method='ffill').fillna(method='bfill')
        
        # 최종 결측치 확인
        remaining_missing = data_filled.isnull().sum().sum()
        print(f" 보간 후 남은 결측치: {remaining_missing}개")
        
        if remaining_missing > 0:
            print(" 여전히 결측치가 있습니다. 해당 행 제거...")
            print("결측치가 있는 컬럼들:")
            remaining_nulls = data_filled.isnull().sum()
            for col, null_count in remaining_nulls.items():
                if null_count > 0:
                    print(f"  {col}: {null_count}개")
            
            # 결측치가 있는 행 제거
            data_filled = data_filled.dropna()
            print(f"결측치 제거 후 데이터 크기: {data_filled.shape}")
    else:
        print("결측치가 없습니다!")
        data_filled = combined_data.copy()
    
    # 완전한 데이터만 사용
    final_data = data_filled.copy()
    
    print(f"\n 최종 데이터 크기: {final_data.shape}")
    if len(final_data) > 0:
        print(f" 최종 기간: {final_data.index[0]} ~ {final_data.index[-1]}")
        
        # 기초 통계 확인
        print("\n 기초 통계 (처음 5개 컬럼):")
        print(final_data.iloc[:, :5].describe().round(2))
        
        print(f"\n 각 컬럼별 데이터 요약:")
        for col in final_data.columns:
            data_count = len(final_data[col])
            data_mean = final_data[col].mean()
            data_std = final_data[col].std()
            print(f"  {col}: {data_count}개, 평균={data_mean:.2f}, 표준편차={data_std:.2f}")
    else:
        print(" 최종 데이터가 비어있습니다.")
    
else:
    print(" 처리할 데이터가 없습니다.")
    final_data = pd.DataFrame()

🔄 결측치 처리 중...
 결측치 현황:
전체 데이터 크기: (186, 22)
총 결측치: 0개
결측치가 없습니다!

 최종 데이터 크기: (186, 22)
 최종 기간: 2010-01 ~ 2025-06

 기초 통계 (처음 5개 컬럼):
       Gold_Price  US_M2_Money_Supply  US_CPI   Gold_Volume  \
count      186.00              186.00  186.00  1.860000e+02   
mean      1593.92            14849.13  256.27  2.025577e+08   
std        440.58             4566.86   30.35  9.028885e+07   
min       1069.95             8478.00  216.69  9.735590e+07   
25%       1266.34            10999.12  233.75  1.408006e+08   
50%       1483.25            13752.65  246.59  1.789454e+08   
75%       1805.52            20672.80  273.43  2.353245e+08   
max       3353.92            22021.40  322.56  7.685474e+08   

       Gold_Reserves_Gold_Reserves_United_States  
count                                     186.00  
mean                                     8129.73  
std                                       153.46  
min                                      7707.35  
25%                                      80